In [1]:
import os
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
from os.path import join
from scipy.optimize import curve_fit
import pandas as pd
from tqdm import tqdm
crop='swh'
month=[1, 2,10, 11, 12]
tpye='Irr'
acc='irr'
hemisphere='sou'
# lat=85;lon=608
fp=r'/pb1/home/wanghaiqing@iga.ac.cn/gwsp3'
ds = xr.open_dataset(join(fp, f'crop_daily_tavg_{tpye}_daily_{crop}_{hemisphere}_1.nc'),decode_times=False)[f'crop_daily_tavg_{tpye}']
time_raw = ds.time.values 
start_date = pd.Timestamp('1901-01-01')
time_index = [start_date + pd.Timedelta(days=int(day)) for day in time_raw]
tas1 = ds.assign_coords(time=('time', time_index))
tas1=tas1.sel(time=tas1['time.month'].isin(month))
ds = xr.open_dataset(join(fp, f'crop_daily_tavg_{tpye}_daily_{crop}_{hemisphere}_2.nc'),decode_times=False)[f'crop_daily_tavg_{tpye}']
tas2 = ds.assign_coords(time=('time', time_index))
tas2=tas2.sel(time=tas2['time.month'].isin(month))
tas=np.zeros_like(tas1.values)
tas2.values
for i in tqdm(range(30)):
    tas[:,i*12:(i+1)*12,:]=tas1[:,i*12:(i+1)*12,:]+tas2[:,i*12:(i+1)*12,:]
tas = xr.DataArray(
    tas, 
    coords={"time": tas1.time, "lat": tas1.lat, "lon": tas1.lon}, 
    dims=["time", "lat", "lon"],
    name="tas"
)

del tas1, tas2

ds = xr.open_dataset(join(fp, f'crop_daily_Precipitation_{tpye}_daily_{crop}_{hemisphere}_1.nc'),decode_times=False)[f'crop_daily_Precipitation_{tpye}']
pr1 = ds.assign_coords(time=('time', time_index))
pr1=pr1.sel(time=pr1['time.month'].isin(month))*1000
ds = xr.open_dataset(join(fp, f'crop_daily_Precipitation_{tpye}_daily_{crop}_{hemisphere}_2.nc'),decode_times=False)[f'crop_daily_Precipitation_{tpye}']
pr2 = ds.assign_coords(time=('time', time_index))
pr2=pr2.sel(time=pr2['time.month'].isin(month))*1000
pr=np.zeros_like(pr1.values)
pr2.values
for i in tqdm(range(30)):
    pr[:,i*12:(i+1)*12,:]=pr1[:,i*12:(i+1)*12,:]+pr2[:,i*12:(i+1)*12,:]
pr = xr.DataArray(
    pr, 
    coords={"time": pr1.time, "lat": pr1.lat, "lon": pr1.lon}, 
    dims=["time", "lat", "lon"],
    name="tas"
)
del pr1, pr2
# ds_eta = xr.open_dataset(join(fp, 'eta_nonIrr_daily_maiz_nor.nc'),decode_times=False).eta_nonIrr 
# eta = ds_eta[:,lat,lon].assign_coords(time=('time', time_index))
# # eta=eta.sel(time=eta['time.month'].isin([5, 6, 7, 8]))*1000
# eta=eta.sel(time=eta['time.month'].isin([5, 6, 7, 8,9])).dropna(dim='time', how='any')*1000

ds = xr.open_dataset(join(fp, f'ratio_et_{tpye}_daily_{crop}_{hemisphere}_1.nc'),decode_times=False)[f'ratio_et_{tpye}']
ratio_et_nonIrr1 = ds.assign_coords(time=('time', time_index))
ratio_et_nonIrr1 = ratio_et_nonIrr1.sel(time=ratio_et_nonIrr1['time.month'].isin(month))
ds = xr.open_dataset(join(fp, f'ratio_et_{tpye}_daily_{crop}_{hemisphere}_2.nc'),decode_times=False)[f'ratio_et_{tpye}']
ratio_et_nonIrr2 = ds.assign_coords(time=('time', time_index))
ratio_et_nonIrr2 = ratio_et_nonIrr2.sel(time=ratio_et_nonIrr2['time.month'].isin(month))
ratio_et_nonIrr=np.zeros_like(ratio_et_nonIrr1.values)
ratio_et_nonIrr2.values
for i in tqdm(range(30)):
    ratio_et_nonIrr[:,i*12:(i+1)*12,:]=ratio_et_nonIrr1[:,i*12:(i+1)*12,:]+ratio_et_nonIrr2[:,i*12:(i+1)*12,:]
ratio_et_nonIrr = xr.DataArray(
    ratio_et_nonIrr, 
    coords={"time": ratio_et_nonIrr1.time, "lat": ratio_et_nonIrr1.lat, "lon": ratio_et_nonIrr1.lon}, 
    dims=["time", "lat", "lon"],
    name="tas"
)
del ratio_et_nonIrr2,ratio_et_nonIrr1
frac_path=r'/pb1/home/wanghaiqing@iga.ac.cn/crop_fraction'
frac=xr.open_dataset(join(frac_path,f'{crop}_{hemisphere}_{acc}.nc')).__xarray_dataarray_variable__
frac=frac.interp(lat=ds.lat,lon=ds.lon)
# frac2=xr.open_dataset(join(frac_path,f'{crop}_{hemisphere}_{acc}.nc')).__xarray_dataarray_variable__.interp(lat=ds.lat,lon=ds.lon)

ds = xr.open_dataset(join(fp, f'PotET_crop_{tpye}_daily_{crop}_{hemisphere}_1.nc'),decode_times=False)[f'PotET_crop_{tpye}']
pet1 = ds.assign_coords(time=('time', time_index))
pet1=pet1.sel(time=pet1['time.month'].isin(month))*1000
ds = xr.open_dataset(join(fp, f'PotET_crop_{tpye}_daily_{crop}_{hemisphere}_2.nc'),decode_times=False)[f'PotET_crop_{tpye}']
pet2 = ds.assign_coords(time=('time', time_index))
pet2=pet2.sel(time=pet2['time.month'].isin(month))*1000
pet=np.zeros_like(pet1.values)
pet2.values
for i in tqdm(range(30)):
    pet[:,i*12:(i+1)*12,:]=(pet1[:,i*12:(i+1)*12,:]+pet2[:,i*12:(i+1)*12,:])/frac[i*12:(i+1)*12,:].values
pet = xr.DataArray(
    pet, 
    coords={"time": pet1.time, "lat": pet1.lat, "lon": pet1.lon}, 
    dims=["time", "lat", "lon"],
    name="tas"
)
del pet1,pet2,ds

# # ds_yr = xr.open_dataset(join(r'F:\CWatM\cwatm_output\global_future\GWSP3_W5E5', 'ky_ratio_et_nonIrr_daily_maiz_nor.nc'),decode_times=False).ky_ratio_et_nonIrr
# # yr = ds_yr[:,lat,lon].assign_coords(time=('time', time_index))
# # # pet=pet.sel(time=pet['time.month'].isin([5, 6, 7, 8]))*1000
# # yr=yr.sel(time=yr['time.month'].isin([5, 6, 7, 8,9])).dropna(dim='time', how='any')
# # pet=eta/ratio_et_nonIrr
dr=(pr-pet)
del pet
dr_min=dr.min(dim='time')
dr_max=dr.max(dim='time')

100%|██████████| 30/30 [00:21<00:00,  1.38it/s]


In [4]:
import numpy as np
from tqdm import tqdm
import xarray as xr  # 确保已经导入xarray

latitudes = dr.coords['lat']
longitudes = dr.coords['lon']
bins = np.arange(0, 100.1, 0.1)

# 创建一个与 dr 相同形状的二维数组
shape = dr[0].shape  # 获取 dr 的形状
drought = np.full(shape, np.nan)  # 第一个变量，初始化为 NaN
flood = np.full(shape, np.nan)  # 第二个变量，初始化为 NaN
key=12
for ii in tqdm(range(30)):
    if np.isnan(np.nanmax(ratio_et_nonIrr[0, ii*key:(ii + 1) * key, :])) or np.nanmax(ratio_et_nonIrr[0, ii*key:(ii + 1) * key, :]) == 0:
        continue
    else:
        rat = ratio_et_nonIrr[:, ii*key:(ii + 1) * key, :]
        
        dr_idx = 100 * (dr[:, ii*key:(ii + 1) * key, :] - dr_min[ii*key:(ii + 1) * key, :]) / (dr_max[ii*key:(ii + 1) * key, :] - dr_min[ii*key:(ii + 1) * key, :])
        # dr_idx = dr[:, ii*30:(ii + 1) * 30, :] - dr_min[ii*30:(ii + 1) * 30, :]
        bins = np.arange(0, 100 + 1, 1)  # 以1为间隔划分区间
        means = np.full([len(bins) - 1, len(dr_idx.lat), len(dr_idx.lon)], np.nan) 
        dr_idx_flat = dr_idx.values.flatten()
        rat_flat = rat.values.flatten()

        for i in range(len(bins) - 1):
            # 筛选出当前区间的数据
            mask = (dr_idx_flat >= bins[i]) & (dr_idx_flat < bins[i + 1])
            rat_in_bin = np.where(mask, rat_flat, np.nan).reshape(rat.shape)

            # 使用布尔数组进行索引
            masked_rat_da = xr.DataArray(
                rat_in_bin,
                dims=rat.dims,
                coords=rat.coords
            )

            # 计算均值
            means[i] = masked_rat_da.quantile(0.95, dim="time", skipna=True) * np.sum((rat_in_bin > 0), axis=0) / 100

        for la in tqdm(range(len(dr_idx.lat)),leave=False):
            for lo in range(len(dr_idx.lon)):
                if np.all(np.isnan(means[:, la, lo])):  # 检查所有均值是否为 NaN
                    continue
                
                d = []
                f = []

                mean_rat = np.nanmean(rat[:, la, lo])  # 计算有效的均值
                for i in range(1, len(means[:, la, lo])):  # 从1开始以便可以访问means[i-1]
                    if not np.isnan(means[i, la, lo]) and not np.isnan(means[i - 1, la, lo]):  # 确保当前和上一个均值不为 NaN
                        if means[i, la, lo] > mean_rat and bins[i - 1] < 20 and means[i - 1, la, lo] < mean_rat:
                            d.append(bins[i - 1] + 0.5)
                        if means[i, la, lo] < mean_rat and bins[i - 1] > 20 and means[i - 1, la, lo] > mean_rat:
                            f.append(bins[i - 1] + 0.5)
                
                drought[ii * key + la, lo] = np.min(d) if d else np.nan
                flood[ii * key + la, lo] = np.max(f) if f else np.nan
result = xr.Dataset({
    'drought': (( 'lat', 'lon'), drought),
    'flood': (( 'lat', 'lon'), flood),
}, coords={
    # 'time': dr.coords['time'],
    'lat': dr.lat,
    'lon': dr.lon
})
result.to_netcdf(join(r'/pb1/home/wanghaiqing@iga.ac.cn/results',f'{crop}_{hemisphere}_{acc}_threshold.nc'))

  0%|          | 0/30 [00:00<?, ?it/s]/tmp/ipykernel_200540/4159160865.py:15: RuntimeWarning: All-NaN axis encountered
  if np.isnan(np.nanmax(ratio_et_nonIrr[0, ii*key:(ii + 1) * key, :])) or np.nanmax(ratio_et_nonIrr[0, ii*key:(ii + 1) * key, :]) == 0:
/usr/lib/python3/dist-packages/numpy/lib/nanfunctions.py:1577: RuntimeWarning: All-NaN slice encountered
  result = np.apply_along_axis(_nanquantile_1d, axis, a, q,

100%|██████████| 30/30 [07:13<00:00, 14.44s/it][A


In [5]:
import numpy as np
from tqdm import tqdm
import xarray as xr 

shape = dr[0].shape 
drought_tas = np.full(shape, np.nan)  
drought_pr = np.full(shape, np.nan)  
flood_tas = np.full(shape, np.nan)  
flood_pr = np.full(shape, np.nan)  
drought=xr.open_dataset(join(r'/pb1/home/wanghaiqing@iga.ac.cn/results',f'{crop}_{hemisphere}_{acc}_threshold.nc')).drought
flood=xr.open_dataset(join(r'/pb1/home/wanghaiqing@iga.ac.cn/results',f'{crop}_{hemisphere}_{acc}_threshold.nc')).flood
key=30
for ii in tqdm(range(12)):
    if np.isnan(np.nanmax(ratio_et_nonIrr[0, ii*key:(ii + 1) * key, :])) or np.nanmax(ratio_et_nonIrr[0, ii*key:(ii + 1) * key, :]) == 0:
        continue
    else:
        pr_idx = pr[:, ii*key:(ii + 1) * key, :]
        tas_idx = tas[:, ii*key:(ii + 1) * key, :]
        flood_idx= flood[ ii*key:(ii + 1) * key, :]
        drought_idx= drought[ ii*key:(ii + 1) * key, :]
        dr_idx = 100 * (dr[:, ii*key:(ii + 1) * key, :] - dr_min[ii*key:(ii + 1) * key, :]) / (dr_max[ii*key:(ii + 1) * key, :] - dr_min[ii*key:(ii + 1) * key, :])
        # dr_idx_mm.isel(time=(flood_idx.where(~np.isnan(flood_idx), np.inf)).argmin(dim='time'))
        flood_tas[ii*key:(ii + 1) * key, :] = (tas_idx.where((dr_idx >= flood_idx))).mean(dim='time')
        flood_pr[ii*key:(ii + 1) * key, :] = (pr_idx.where((dr_idx >= flood_idx))).mean(dim='time')
        drought_tas[ii*key:(ii + 1) * key, :] = (tas_idx.where((dr_idx <= drought_idx))).mean(dim='time')
        drought_pr[ii*key:(ii + 1) * key, :] = (pr_idx.where((dr_idx <= drought_idx))).mean(dim='time')
result = xr.Dataset({
    'drought_tas': (( 'lat', 'lon'), drought_tas),
    'drought_pr': (( 'lat', 'lon'), drought_pr),
    'flood_pr': (( 'lat', 'lon'), flood_pr),
    'flood_tas': (( 'lat', 'lon'), flood_tas),
}, coords={
    # 'time': dr.coords['time'],
    'lat': dr.lat,
    'lon': dr.lon
})
result.to_netcdf(join(r'/pb1/home/wanghaiqing@iga.ac.cn/results',f'{crop}_{hemisphere}_{acc}_threshold_climate.nc'))

 75%|███████▌  | 9/12 [00:13<00:05,  1.71s/it]/tmp/ipykernel_200540/2120903033.py:14: RuntimeWarning: All-NaN axis encountered
  if np.isnan(np.nanmax(ratio_et_nonIrr[0, ii*key:(ii + 1) * key, :])) or np.nanmax(ratio_et_nonIrr[0, ii*key:(ii + 1) * key, :]) == 0:
100%|██████████| 12/12 [00:13<00:00,  1.14s/it]


In [6]:
import numpy as np
from tqdm import tqdm
import xarray as xr  # 确保已经导入xarray

# 创建一个与 dr 相同形状的二维数组
shape = dr[0].shape  # 获取 dr 的形状
drought_mm = np.full(shape, np.nan)  # 第一个变量，初始化为 NaN
flood_mm = np.full(shape, np.nan)  # 第二个变量，初始化为 NaN
drought=xr.open_dataset(join(r'/pb1/home/wanghaiqing@iga.ac.cn/results',f'{crop}_{hemisphere}_{acc}_threshold.nc')).drought
flood=xr.open_dataset(join(r'/pb1/home/wanghaiqing@iga.ac.cn/results',f'{crop}_{hemisphere}_{acc}_threshold.nc')).flood
key=30
for ii in tqdm(range(12)):
    if np.isnan(np.nanmax(ratio_et_nonIrr[0, ii*key:(ii + 1) * key, :])) or np.nanmax(ratio_et_nonIrr[0, ii*key:(ii + 1) * key, :]) == 0:
        continue
    else:
        
        dr_idx_mm = dr[:, ii*key:(ii + 1) * key, :] - dr_min[ii*key:(ii + 1) * key, :]
        dr_idx = 100 * (dr[:, ii*key:(ii + 1) * key, :] - dr_min[ii*key:(ii + 1) * key, :]) / (dr_max[ii*key:(ii + 1) * key, :] - dr_min[ii*key:(ii + 1) * key, :])
        flood_idx= np.abs(dr_idx-flood[ ii*key:(ii + 1) * key, :])
        drought_idx= np.abs(dr_idx-drought[ ii*key:(ii + 1) * key, :])
        
        flood_mm[ii*key:(ii + 1) * key, :] = dr_idx_mm.isel(time=(flood_idx.where(~np.isnan(flood_idx), np.inf)).argmin(dim='time'))
        drought_mm[ii*key:(ii + 1) * key, :] = dr_idx_mm.isel(time=(drought_idx.where(~np.isnan(drought_idx), np.inf)).argmin(dim='time'))
flood_mm=np.where(flood>0,flood_mm,np.nan)
drought_mm=np.where(flood>0,drought_mm,np.nan)
result = xr.Dataset({
    'flood_mm': (( 'lat', 'lon'), flood_mm),
    'drought_mm': (( 'lat', 'lon'), drought_mm),
}, coords={
    # 'time': dr.coords['time'],
    'lat': dr.lat,
    'lon': dr.lon
})
result.to_netcdf(join(r'/pb1/home/wanghaiqing@iga.ac.cn/results',f'{crop}_{hemisphere}_{acc}_threshold_mm.nc'))

 75%|███████▌  | 9/12 [00:09<00:03,  1.26s/it]/tmp/ipykernel_200540/3819057257.py:13: RuntimeWarning: All-NaN axis encountered
  if np.isnan(np.nanmax(ratio_et_nonIrr[0, ii*key:(ii + 1) * key, :])) or np.nanmax(ratio_et_nonIrr[0, ii*key:(ii + 1) * key, :]) == 0:
100%|██████████| 12/12 [00:09<00:00,  1.23it/s]


In [7]:
import numpy as np
from tqdm import tqdm
import xarray as xr  # 确保已经导入xarray

latitudes = dr.coords['lat']
longitudes = dr.coords['lon']
bins = np.arange(0, 100.1, 0.1)

# 创建一个与 dr 相同形状的二维数组
shape = dr[0].shape  # 获取 dr 的形状
drought_tas = np.full(shape, np.nan)  # 第一个变量，初始化为 NaN
drought_pr = np.full(shape, np.nan)  # 第一个变量，初始化为 NaN
flood_tas = np.full(shape, np.nan)  # 第一个变量，初始化为 NaN
flood_pr = np.full(shape, np.nan)  # 第一个变量，初始化为 NaN
drought=xr.open_dataset(join(r'/pb1/home/wanghaiqing@iga.ac.cn/results',f'{crop}_{hemisphere}_{acc}_threshold.nc')).drought
flood=xr.open_dataset(join(r'/pb1/home/wanghaiqing@iga.ac.cn/results',f'{crop}_{hemisphere}_{acc}_threshold.nc')).flood
key=30
for ii in tqdm(range(12)):
    if np.isnan(np.nanmax(ratio_et_nonIrr[0, ii*key:(ii + 1) * key, :])) or np.nanmax(ratio_et_nonIrr[0, ii*key:(ii + 1) * key, :]) == 0:
        continue
    else:
        pr_idx = pr[:, ii*key:(ii + 1) * key, :]
        tas_idx = tas[:, ii*key:(ii + 1) * key, :]
        flood_idx= flood[ ii*key:(ii + 1) * key, :]
        drought_idx= drought[ ii*key:(ii + 1) * key, :]
        dr_idx = 100 * (dr[:, ii*key:(ii + 1) * key, :] - dr_min[ii*key:(ii + 1) * key, :]) /\
        (dr_max[ii*key:(ii + 1) * key, :] - dr_min[ii*key:(ii + 1) * key, :])
        
        flood_tas[ii*key:(ii + 1) * key, :] = (tas_idx.where((dr_idx >= flood_idx))).sum(dim='time')
        flood_pr[ii*key:(ii + 1) * key, :] = (pr_idx.where((dr_idx >= flood_idx))).sum(dim='time')
        drought_tas[ii*key:(ii + 1) * key, :] = (tas_idx.where((dr_idx <= drought_idx))).sum(dim='time')
        drought_pr[ii*key:(ii + 1) * key, :] = (pr_idx.where((dr_idx <= drought_idx))).sum(dim='time')
result = xr.Dataset({
    'drought_tas': (( 'lat', 'lon'), drought_tas),
    'drought_pr': (( 'lat', 'lon'), drought_pr),
    'flood_pr': (( 'lat', 'lon'), flood_pr),
    'flood_tas': (( 'lat', 'lon'), flood_tas),
}, coords={
    # 'time': dr.coords['time'],
    'lat': dr.lat,
    'lon': dr.lon
})
result.to_netcdf(join(r'/pb1/home/wanghaiqing@iga.ac.cn/results',f'{crop}_{hemisphere}_{acc}_threshold_climate_accrue.nc'))

 75%|███████▌  | 9/12 [00:10<00:04,  1.39s/it]/tmp/ipykernel_200540/1614313395.py:19: RuntimeWarning: All-NaN axis encountered
  if np.isnan(np.nanmax(ratio_et_nonIrr[0, ii*key:(ii + 1) * key, :])) or np.nanmax(ratio_et_nonIrr[0, ii*key:(ii + 1) * key, :]) == 0:
100%|██████████| 12/12 [00:10<00:00,  1.11it/s]


In [10]:
import os
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
from os.path import join
from scipy.optimize import curve_fit
import pandas as pd
from tqdm import tqdm
crop='maiz'
month=[1, 2,3, 11, 12]
tpye='nonIrr'
acc='rf'
hemisphere='sou'
# lat=85;lon=608
fp=r'/pb1/home/wanghaiqing@iga.ac.cn/gwsp3'


ds = xr.open_dataset(join(fp, f'ratio_et_{tpye}_daily_{crop}_{hemisphere}_1.nc'),decode_times=False)[f'ratio_et_{tpye}']
time_raw = ds.time.values 
start_date = pd.Timestamp('1901-01-01')
time_index = [start_date + pd.Timedelta(days=int(day)) for day in time_raw]
ratio_et_nonIrr1 = ds.assign_coords(time=('time', time_index))
ratio_et_nonIrr1 = ratio_et_nonIrr1.sel(time=ratio_et_nonIrr1['time.month'].isin(month))
ds = xr.open_dataset(join(fp, f'ratio_et_{tpye}_daily_{crop}_{hemisphere}_2.nc'),decode_times=False)[f'ratio_et_{tpye}']
ratio_et_nonIrr2 = ds.assign_coords(time=('time', time_index))
ratio_et_nonIrr2 = ratio_et_nonIrr2.sel(time=ratio_et_nonIrr2['time.month'].isin(month))
ratio_et_nonIrr=np.zeros_like(ratio_et_nonIrr1.values)
ratio_et_nonIrr2.values
for i in tqdm(range(30)):
    ratio_et_nonIrr[:,i*12:(i+1)*12,:]=ratio_et_nonIrr1[:,i*12:(i+1)*12,:]+ratio_et_nonIrr2[:,i*12:(i+1)*12,:]
ratio_et_nonIrr = xr.DataArray(
    ratio_et_nonIrr, 
    coords={"time": ratio_et_nonIrr1.time, "lat": ratio_et_nonIrr1.lat, "lon": ratio_et_nonIrr1.lon}, 
    dims=["time", "lat", "lon"],
    name="tas"
)
del ratio_et_nonIrr2,ratio_et_nonIrr1
ratio_et_nonIrr.mean(dim='time').to_netcdf(join(r'/pb1/home/wanghaiqing@iga.ac.cn/future',f'{crop}_{hemisphere}_{acc}_ration_mean.nc'))

100%|██████████| 30/30 [00:04<00:00,  6.32it/s]


### frequency

In [3]:
import os
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
from os.path import join
from scipy.optimize import curve_fit
import pandas as pd
from tqdm import tqdm
crop='wwh'
month=[1,2,3,4,5,10,11,12]
tpye='nonIrr'
acc='rf'
hemisphere='nor'
# lat=85;lon=608
fp=r'/pb1/home/wanghaiqing@iga.ac.cn/gwsp3'
ds = xr.open_dataset(join(fp, f'crop_daily_Precipitation_{tpye}_daily_{crop}_{hemisphere}_1.nc'),decode_times=False)[f'crop_daily_Precipitation_{tpye}']
time_raw = ds.time.values 
start_date = pd.Timestamp('1901-01-01')
time_index = [start_date + pd.Timedelta(days=int(day)) for day in time_raw]
pr1 = ds.assign_coords(time=('time', time_index))
pr1=pr1.sel(time=pr1['time.month'].isin(month))*1000
ds = xr.open_dataset(join(fp, f'crop_daily_Precipitation_{tpye}_daily_{crop}_{hemisphere}_2.nc'),decode_times=False)[f'crop_daily_Precipitation_{tpye}']
pr2 = ds.assign_coords(time=('time', time_index))
pr2=pr2.sel(time=pr2['time.month'].isin(month))*1000
pr=np.zeros_like(pr1.values)
pr2.values
for i in tqdm(range(30)):
    pr[:,i*12:(i+1)*12,:]=pr1[:,i*12:(i+1)*12,:]+pr2[:,i*12:(i+1)*12,:]
pr = xr.DataArray(
    pr, 
    coords={"time": pr1.time, "lat": pr1.lat, "lon": pr1.lon}, 
    dims=["time", "lat", "lon"],
    name="tas"
)
del pr1, pr2

frac_path=r'/pb1/home/wanghaiqing@iga.ac.cn/crop_fraction'
frac=xr.open_dataset(join(frac_path,f'{crop}_{hemisphere}_{acc}.nc')).__xarray_dataarray_variable__
frac=frac.interp(lat=ds.lat,lon=ds.lon)
# frac2=xr.open_dataset(join(frac_path,f'{crop}_{hemisphere}_{acc}.nc')).__xarray_dataarray_variable__.interp(lat=ds.lat,lon=ds.lon)

ds = xr.open_dataset(join(fp, f'PotET_crop_{tpye}_daily_{crop}_{hemisphere}_1.nc'),decode_times=False)[f'PotET_crop_{tpye}']
pet1 = ds.assign_coords(time=('time', time_index))
pet1=pet1.sel(time=pet1['time.month'].isin(month))*1000
ds = xr.open_dataset(join(fp, f'PotET_crop_{tpye}_daily_{crop}_{hemisphere}_2.nc'),decode_times=False)[f'PotET_crop_{tpye}']
pet2 = ds.assign_coords(time=('time', time_index))
pet2=pet2.sel(time=pet2['time.month'].isin(month))*1000
pet=np.zeros_like(pet1.values)
pet2.values
for i in tqdm(range(30)):
    pet[:,i*12:(i+1)*12,:]=(pet1[:,i*12:(i+1)*12,:]+pet2[:,i*12:(i+1)*12,:])/frac[i*12:(i+1)*12,:].values
pet = xr.DataArray(
    pet, 
    coords={"time": pet1.time, "lat": pet1.lat, "lon": pet1.lon}, 
    dims=["time", "lat", "lon"],
    name="tas"
)
del pet1,pet2,ds

dr=(pr-pet)
del pet


100%|██████████| 30/30 [00:38<00:00,  1.29s/it]


In [4]:
drought=xr.open_dataset(join(r'/pb1/home/wanghaiqing@iga.ac.cn/results',f'{crop}_{hemisphere}_{acc}_threshold_mm.nc')).drought_mm
flood=xr.open_dataset(join(r'/pb1/home/wanghaiqing@iga.ac.cn/results',f'{crop}_{hemisphere}_{acc}_threshold_mm.nc')).flood_mm

condition = dr >= flood
count_flood = np.sum(condition, axis=0)
condition = dr <= drought
count_drought = np.sum(condition, axis=0)
result = xr.Dataset({
    'count_drought': (( 'lat', 'lon'), count_drought.values),
    'count_flood': (( 'lat', 'lon'), count_flood.values),
}, coords={
    # 'time': dr.coords['time'],
    'lat': dr.lat,
    'lon': dr.lon
})
result.to_netcdf(join(r'/pb1/home/wanghaiqing@iga.ac.cn/results',f'{crop}_{hemisphere}_{acc}_threshold_climate_frequency.nc'))